## Les sorties structurées

### Cas d'un booléen

In [8]:
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai.chat_models import ChatMistralAI

class Answer(BaseModel):
    answer: bool

prompt_answer = [
    ("system", "Tu es un assistant chargé de répondre un booléen (True ou False) à la question d'un utilisateur."),
    ('human', "{question}")
]

prompt_answer_template = ChatPromptTemplate.from_messages(prompt_answer)
llm = ChatMistralAI(model="mistral-large-latest", temperature=0)
chain = prompt_answer_template | llm.with_structured_output(schema=Answer)

def repond(question):
    return chain.invoke({"question": question}).answer
    
for question in ["Noël est en hiver", "Il pleut quand il pleut pas"]:
    print(question)
    reponse = repond(question)
    print(f"Réponse : {reponse} (type : {type(reponse)})")
    print()


Noël est en hiver
Réponse : True (type : <class 'bool'>)

Il pleut quand il pleut pas
Réponse : False (type : <class 'bool'>)



### Cas d'une classe

In [9]:
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai.chat_models import ChatMistralAI

tasks = ["Répondre à une nouvelle question", "Fournir plus d'éléments à la question précédente"]

class NextTask(BaseModel):
    """Utilise toujours cet outil pour structurer ta réponse to the user."""
    action: str = Field(..., 
                        enum=tasks,
                        description="La prochaine action à mener")

prompt_message = [
    ("system", "Tu es un assistant chargé de classifier la demande d'un utilisateur parmi une "
               "liste réduite d'actions à mener en tant que chatbot. Tu dois déterminer la "
               "prochaine action à mener."),
    ('human', "{text}")
]

prompt = ChatPromptTemplate.from_messages(prompt_message)
llm = ChatMistralAI(model='mistral-large-latest', temperature=0)
chain = prompt | llm.with_structured_output(schema=NextTask)

for text in ["Peux-tu m'en dire plus", "Que sont les PPV ?"]:
    print(text)
    print(chain.invoke({"text": text}))
    print()

Peux-tu m'en dire plus
action="Fournir plus d'éléments à la question précédente"

Que sont les PPV ?
action='Répondre à une nouvelle question'



### Cas d'un entier

In [10]:
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai.chat_models import ChatMistralAI

class TonMessage(BaseModel):
    """Évaluation du ton du message de l'utilisateur."""
    note_ton: int = Field(
        ..., 
        ge=1,
        le=5,
        description="Note attribuée au ton du message : 1 pour neutre, 5 pour très aimable"
    )

prompt_message = [
    ("system", "Tu es un assistant chargé d'évaluer le ton d'un message donné par l'utilisateur. "
               "Attribue une note de 1 à 5 au ton du message, où 1 signifie neutre et 5 signifie très aimable."),
    ('human', "{text}")
]

prompt = ChatPromptTemplate.from_messages(prompt_message)
llm = ChatMistralAI(model='mistral-large-latest', temperature=0)
chain = prompt | llm.with_structured_output(schema=TonMessage)

messages = [
    "Bonjour, pourrais-tu m'aider s'il te plaît ?",
    "J'ai besoin de ça immédiatement.",
    "Merci beaucoup pour ton aide précieuse !"
]

for text in messages:
    print(f"Message : {text}")
    print(chain.invoke({"text": text}))
    print()

Message : Bonjour, pourrais-tu m'aider s'il te plaît ?
note_ton=4

Message : J'ai besoin de ça immédiatement.
note_ton=1

Message : Merci beaucoup pour ton aide précieuse !
note_ton=5



### Pourquoi et comment forcer la sortie du LLM

In [11]:
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.messages import HumanMessage

llm = ChatMistralAI(model="mistral-medium-latest")
message = HumanMessage(content="Peux-tu me traduire ce qui suit, en anglais ?\n\n Quelle est la capitale de l'Albanie ?")
llm.invoke([message])

AIMessage(content='The translation of "Quelle est la capitale de l\'Albanie ?" in English is:\n\n**"What is the capital of Albania?"**', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 26, 'total_tokens': 56, 'completion_tokens': 30}, 'model_name': 'mistral-medium-latest', 'model': 'mistral-medium-latest', 'finish_reason': 'stop'}, id='run--c4b5c2c7-5a43-4f1d-a93e-0ed2170a7818-0', usage_metadata={'input_tokens': 26, 'output_tokens': 30, 'total_tokens': 56})

In [21]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_mistralai.chat_models import ChatMistralAI
from pydantic import BaseModel, Field

class Translation(BaseModel):
    original_text: str = Field(..., description="The original text before translation in another language")
    original_language: str = Field(..., description="The original language before translation")
    translated_text: str = Field(..., description="The final text after translation in another language")
    translated_language: str = Field(..., description="The language into which the translation must be done")
    
def traduit(texte, langue_source="français", langue_cible="anglais"):
    llm = ChatMistralAI(model_name="mistral-medium-latest")
    prompt = ChatPromptTemplate.from_template("""Je souhaite que tu traduises le texte suivant du {langue_source} vers le {langue_cible}. Ta traduction doit être précise, fluide et naturelle, et préserver parfaitement le sens original. 
    Retourne-moi la réponse sous forme d'objet JSON avec les champs :
      - original_text : le texte original
      - original_language : la langue du texte original
      - translated_text : la traduction du texte
      - translated_language : la langue de la traduction

    Voici le texte à traduire :
    ----
    {texte}""")
    output_parser = StrOutputParser()
    extract_translation = RunnableLambda(lambda translation: translation.translated_text)
    chain0 = prompt | llm.with_structured_output(Translation) | extract_translation
    return chain0.invoke({"langue_source": langue_source,
                            "langue_cible": "anglais",
                            "texte": texte      
                            })

print(traduit("Quelle est la capitale de l'Albanie"))

What is the capital of Albania


## Des sorties structurées aux prémices d'un raisonnement

In [13]:
from pydantic import BaseModel
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai.chat_models import ChatMistralAI

class Etape(BaseModel):
    explication: str
    sortie: str

class MathReponse(BaseModel):
    etapes: list[Etape]
    reponse_finale: str

prompt_answer = [
    ("system", "Tu es un professeur de mathématiques très pédagogue."),
    ('human', "{exercice}")
]

prompt_answer_template = ChatPromptTemplate.from_messages(prompt_answer)
llm = ChatMistralAI(model="mistral-large-latest", temperature=0)
chain = prompt_answer_template | llm.with_structured_output(schema=MathReponse)

In [14]:
explications = chain.invoke({"exercice": "Résous  8x + 31 = 2"})
for etape in explications.etapes:
    print(f"- {etape.explication}")
    print(f"  Le résultat est alors : {etape.sortie}")

print(f"Au final, on trouve : {explications.reponse_finale}")

- Soustrayons 31 des deux côtés de l'équation pour isoler le terme en x.
  Le résultat est alors : 8x + 31 - 31 = 2 - 31
8x = -29
- Divisons les deux côtés par 8 pour résoudre x.
  Le résultat est alors : 8x / 8 = -29 / 8
x = -29/8 ou x = -3,625
Au final, on trouve : x = -3,625
